In [20]:
#this program will apply the bias subtraction,
#flat fielding and subtract the gradient background,
#it will also align the images to the first image in the list

#if you use this code, please cite Oelkers et al. 2015, AJ, 149, 50

#import the relevant libraries for basic tools
import numpy 
import scipy
from scipy import stats
import scipy.ndimage as ndimage
import astropy
from astropy.io import fits
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS
import math
import time
from photutils import centroid_com, centroid_1dg, centroid_2dg

#libraries for image registration
import FITS_tools
from FITS_tools.hcongrid import hcongrid

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join

#import relevant spline libraries
from scipy.interpolate import Rbf

In [21]:
#user = input('Insert the name of your user: ')
user = 'usuario'

In [22]:
import numpy as np
#####UPDATE INFORMATION HERE####
#DO YOU WANT TO FLAT FIELD AND BIAS SUBTRACT?
align = 1# yes = 1 no = 0 to align based on coordinates

#useful directories
rawdir = '/home/'+user+'/Documents/Mestrado/FFIs/testeall/' #directory with the raw images
cdedir = '/home/'+user+'/dia_kepler/mychanges/Python/' #directory where the code 'lives'
clndir = '/home/'+user+'/Documents/Mestrado/FFIs/testeall/clean/'#directory for the cleaned images to be output

#sample every how many pixels? here is 20x16 and 1100/20 = 55 & 1024/16 = 64
pix1 = 64 # UPDATE HERE FOR BACKGROUND SPACING
pix2 = 55

axs1 = 1024 # UPDATE HERE FOR IMAGE AXIS SIZE - number of columns
axs2 = 1100 #number of lines

#sample every how many pixels?
bxs1 =  256 #how big do you want to make the boxes for each image? 
bxs2 = 275
###END UPDATE INFORMATION###

#get the image list and the number of files which need reduction
os.chdir(rawdir) #changes to the raw image direcotory
files = [f for f in glob.glob("*.fits") if isfile(join(rawdir, f))] #gets the relevant files with the proper extension
files.sort()
nfiles = len(files)
os.chdir(cdedir) #changes back to the code directory
print(nfiles)

lop = pix1+pix2
sze = (bxs1/pix1)*(bxs2/pix2)+(bxs1/pix1)+(bxs2/pix2)+1 #size holder for later

hdulist = fits.open(rawdir+files[0])
n_images = len(hdulist)-1

ref, rhead = fits.getdata(rawdir+files[0], header = True,ext=1)

rhead['NAXIS1'] = 1100
rhead['NAXIS2'] = 1024

#creating the matrices that will be populate by the bck, sky values and etc
big_sig = numpy.zeros((axs1,axs2))
background = numpy.zeros((axs1,axs2))
mini_back = numpy.zeros((axs1,axs2))

14


In [23]:
# start = datetime.datetime.now()
for ii in range(0, nfiles):

    hld = files[ii].split('.')
    finnme = hld[0]+'_sa.fits'
    #if (os.path.isfile(clndir+finnme) == 0):
    st = time.time()
    sts = time.strftime("%c")
    print('Now cleaning '+files[ii]+' at '+sts+'.')
    hdu = fits.open(rawdir+files[ii])
    fitslist=[]
    fitslist = fits.HDUList()
    fitslist.append(fits.PrimaryHDU())
    
    for js in range(1,len(hdu)):
        orgimg = hdu[js].data
        header = hdu[js].header
        #w = WCS(header)
        cut = Cutout2D(orgimg,(int(axs2/2),int(axs1/2)),(axs1,axs2))#, wcs=w)
        bigimg = cut.data

        #get the holders ready
        res = numpy.zeros(shape=(axs1, axs2)) #holder for the background 'image'
        bck = numpy.zeros(shape=int((axs1/bxs1)*(axs2/bxs2))) #get the holder for the image backgroudn
        sbk = numpy.zeros(shape=int((axs1/bxs1)*(axs2/bxs2))) #get the holder for the sigma of the image background

        tts = 0
        for oo in range(0, axs2, bxs2):
            for ee in range(0, axs1, bxs1):
                img = bigimg[ee:ee+bxs1, oo:oo+bxs2] #split the image into small subsections

                #calculate the sky statistics
                cimg, clow, chigh = scipy.stats.sigmaclip(img, low=2.5, high = 2.5) #do a 2.5 sigma clipping
                sky = numpy.median(cimg) #determine the sky value
                sig = numpy.std(cimg) #determine the sigma(sky)

                bck[tts] = sky #insert the image median background
                sbk[tts] = sig #insert the image sigma background

                #create holder arrays for good and bad pixels
                x = numpy.zeros(shape=int(sze))
                y = numpy.zeros(shape=int(sze))
                v = numpy.zeros(shape=int(sze))
                s = numpy.zeros(shape=int(sze))
                nd = int(0)

                #begin the sampling of the "local" sky value
                for jj in range(0, bxs2+pix2, pix2):
                    for kk in range(0,bxs1+pix1, pix1):
                        il = numpy.amax([jj-lop,0])
                        ih = numpy.amin([jj+lop, bxs2-1])
                        jl = numpy.amax([kk-lop, 0])
                        jh = numpy.amin([kk+lop, bxs1-1])
                        c = img[jl:jh, il:ih]

                        #select the median value with clipping
                        cc, cclow, cchigh = scipy.stats.sigmaclip(c, low=2.5, high = 2.5) #sigma clip the background
                        lsky = numpy.median(cc) #the sky background
                        ssky = numpy.std(cc) #sigma of the sky background
                        x[nd] = numpy.amin([jj, bxs2-1]) #determine the pixel to input
                        y[nd] = numpy.amin([kk, bxs1-1]) #determine the pixel to input
                        v[nd] = lsky #median sky
                        s[nd] = ssky #sigma sky
                        nd = nd + 1
                #now we want to remove any possible values which have bad sky values
                rj = numpy.where(v <= 0) #stuff to remove
                kp = numpy.where(v > 0) #stuff to keep

#                 if (len(rj[0]) > 0):
#                     #keep only the good points
#                     xgood = x[kp]
#                     ygood = y[kp]
#                     vgood = v[kp]
#                     sgood = s[kp]
#                     print(ygood)
                    
#                     for jj in range(0, len(rj[0])):
#                         #select the bad point
#                         xbad = x[rj[jj]]
#                         ybad = y[rj[jj]]
#                         print(ybad)
                        
#                         #use the distance formula to get the closest points
#                         rd = math.sqrt((ygood-ybad)**2.+(xgood-xbad)**2.)
#                         #sort the radii
#                         pp = sorted(range(len(rd)), key = lambda k:rd[k])
#                         #use the closest 10 points to get a median
#                         vnear = vgood[pp[0:9]]
#                         ave = numpy.median(vnear)
#                         #insert the good value into the array
#                         v[rj[jj]] = ave

                #now we want to remove any possible values which have bad sigmas
                rjs = numpy.where(s >= 2*sig) #remove
                rj  = rjs[0]
                kps = numpy.where(s < 2*sig)
                kp  = kps[0]

                if (len(rj) > 0):
                    #keep only the good points
                    xgood = numpy.array(x[kp])
                    ygood = numpy.array(y[kp])
                    vgood = numpy.array(v[kp])
                    sgood = numpy.array(s[kp])

                    for jj in range(0, len(rj)):
                        #select the bad point
                        xbad = x[rj[jj]]
                        ybad = y[rj[jj]]
                        #print xbad, ybad
                        #use the distance formula to get the closest points
                        rd = numpy.sqrt((xgood-xbad)**2.+(ygood-ybad)**2.)
                        #sort the radii
                        pp = sorted(range(len(rd)), key = lambda k:rd[k])
                        #use the closest 10 points to get a median
                        vnear = vgood[pp[0:9]]
                        ave = numpy.median(vnear)
                        #insert the good value into the array
                        v[rj[jj]] = ave

                #now we interpolate to the rest of the image with a thin-plate spline	
                xi = numpy.linspace(0, bxs2-1, bxs2)
                yi = numpy.linspace(0, bxs1-1, bxs1)
                XI, YI = numpy.meshgrid(xi, yi)
                rbf = Rbf(x, y, v, function = 'thin-plate', smooth = 0.0)
                reshld = rbf(XI, YI)

                #now add the values to the residual image
                res[ee:ee+bxs1, oo:oo+bxs2] = reshld
                tts = tts+1
        #get the median background
        mbck = numpy.median(bck)
        sbck = numpy.median(sbk)
         #subtract the sky gradient and add back the median background
        sub = bigimg-res
        sub = sub + mbck

         #update the header
#         header['CTYPE1'] = rhead['CTYPE1']
#         header['CTYPE2'] = rhead['CTYPE2']
#         header['CRVAL1'] = rhead['CRVAL1']
#         header['CRVAL2'] = rhead['CRVAL2']
#         header['CRPIX1'] = rhead['CRPIX1']
#         header['CRPIX2'] = rhead['CRPIX2']
#         header['CD1_1'] = rhead['CD1_1']
#         header['CD1_2'] = rhead['CD1_2']
#         header['CD2_1'] = rhead['CD2_1']
#         header['CD2_2'] = rhead['CD2_2']
        

        #update the header
        header['medback'] = mbck
        header['sigback'] = sbck
        header['bksub'] = 'yes'

        if (align == 1):
            header['align'] = 'yes'
#         algn = hcongrid(sub, header, rhead)
#         print(type(algn))
#         print(type(header))
#         print(type(rhead))
        fitslist.append(fits.ImageHDU(sub,header = header))
#         print(algn)
        print(js)
#         shd = fits.PrimaryHDU(algn, header=header)
    fitslist.writeto(clndir+finnme, overwrite=True)
       #stop the watch
    fn = time.time()
    print('Background subtraction for '+files[ii]+' finished in '+str(fn-st)+'s.')

print('All done!')

Now cleaning kplr2009114174833_ffi-cal.fits at Tue Aug 27 16:00:24 2019.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
Background subtraction for kplr2009114174833_ffi-cal.fits finished in 321.9453544616699s.
Now cleaning kplr2009114204835_ffi-cal.fits at Tue Aug 27 16:05:46 2019.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
Background subtraction for kplr2009114204835_ffi-cal.fits finished in 319.64400029182434s.
Now cleaning kplr2009115002613_ffi-cal.fits at Tue Aug 27 16:11:06 2019.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
